In [1]:
import xarray as xr
import pandas as pd
import numpy as np

In [2]:
export_folder = "/Users/ash/Documents/Workspaces/Data/phd/SEAPODYM_LMTL/2024-12-13_-_Constant_fields_no_transport/data"

My data is not working in BIOMASS computation process with LMTL for now.


In [3]:
# bio = xr.load_dataset(
#     "/Users/ash/Documents/Workspaces/Data/cmems/2024-07 - 2x2 - LMTL no transport/cmems_mod_glo_bgc_my_0.083deg-lmtl_PT1D-i_1723016267080.nc"
# )
# physic = xr.load_dataset(
#     "/Users/ash/Documents/Workspaces/Data/cmems/2024-07 - 2x2 - LMTL no transport/cmems_mod_glo_bgc_my_0.083deg-lmtl-Fphy_PT1D-i_1723015794520.nc"
# )
# data = xr.merge([bio, physic])[["T", "U", "V", "npp"]]
# attrs = data["depth"].attrs
# data = data.assign_coords({"depth": [0, 1, 2]})
# data.depth.attrs = attrs
# data

Instead I use the forcing data from MERCATOR and set the values to constants.


In [4]:
data = xr.open_mfdataset(
    "/Users/ash/Documents/Workspaces/Data/configurations_seapodym/lmtl/4_data_integrated/4_2_epi_upmeso_daily/*.nc"
).load()
data = data.rename({"u": "U", "v": "V", "temperature": "T"})
data = data[["T", "U", "V", "npp"]]
data = data.resample(time="D").mean()


attrs = data["U"].attrs
data["U"] = xr.full_like(data["U"], 0)
data["U"].attrs = attrs

attrs = data["V"].attrs
data["V"] = xr.full_like(data["V"], 0)
data["V"].attrs = attrs

attrs = data["T"].attrs
TEMPERATURE = 28
data["T"] = xr.full_like(data["T"], TEMPERATURE)
data["T"].attrs = attrs

attrs = data["npp"].attrs
NPP = 300
data["npp"] = xr.full_like(data["npp"], NPP)
data["npp"].attrs = attrs
data["npp"].attrs["units"] = "mg m-2 day-1"

# data = data.sel(latitude=slice(-50, 50))
data

<xarray.Dataset> Size: 610MB
Dimensions:    (latitude: 180, depth: 2, longitude: 360, time: 336)
Coordinates:
  * latitude   (latitude) float64 1kB -89.5 -88.5 -87.5 -86.5 ... 87.5 88.5 89.5
  * depth      (depth) int64 16B 0 1
  * longitude  (longitude) float64 3kB -179.5 -178.5 -177.5 ... 178.5 179.5
  * time       (time) datetime64[ns] 3kB 2000-01-16 2000-01-17 ... 2000-12-16
Data variables:
    T          (time, depth, latitude, longitude) float32 174MB 28.0 ... 28.0
    U          (time, depth, latitude, longitude) float32 174MB 0.0 0.0 ... 0.0
    V          (time, depth, latitude, longitude) float32 174MB 0.0 0.0 ... 0.0
    npp        (time, latitude, longitude) float32 87MB 300.0 300.0 ... 300.0
Attributes:
    comment:  Layer bounds computed from limits

## Sets currents to zero


In [5]:
data["U"] = data["U"].where(data["U"].isnull(), 0)
data["V"] = data["V"].where(data["U"].isnull(), 0)
data

<xarray.Dataset> Size: 610MB
Dimensions:    (latitude: 180, depth: 2, longitude: 360, time: 336)
Coordinates:
  * latitude   (latitude) float64 1kB -89.5 -88.5 -87.5 -86.5 ... 87.5 88.5 89.5
  * depth      (depth) int64 16B 0 1
  * longitude  (longitude) float64 3kB -179.5 -178.5 -177.5 ... 178.5 179.5
  * time       (time) datetime64[ns] 3kB 2000-01-16 2000-01-17 ... 2000-12-16
Data variables:
    T          (time, depth, latitude, longitude) float32 174MB 28.0 ... 28.0
    U          (time, depth, latitude, longitude) float32 174MB 0.0 0.0 ... 0.0
    V          (time, depth, latitude, longitude) float32 174MB 0.0 0.0 ... 0.0
    npp        (time, latitude, longitude) float32 87MB 300.0 300.0 ... 300.0
Attributes:
    comment:  Layer bounds computed from limits

## Sets temperature to constant value


In [6]:
TEMPERATURE = 28
data["T"] = data["T"].where(data["T"].isnull(), TEMPERATURE)
data

<xarray.Dataset> Size: 610MB
Dimensions:    (latitude: 180, depth: 2, longitude: 360, time: 336)
Coordinates:
  * latitude   (latitude) float64 1kB -89.5 -88.5 -87.5 -86.5 ... 87.5 88.5 89.5
  * depth      (depth) int64 16B 0 1
  * longitude  (longitude) float64 3kB -179.5 -178.5 -177.5 ... 178.5 179.5
  * time       (time) datetime64[ns] 3kB 2000-01-16 2000-01-17 ... 2000-12-16
Data variables:
    T          (time, depth, latitude, longitude) float32 174MB 28.0 ... 28.0
    U          (time, depth, latitude, longitude) float32 174MB 0.0 0.0 ... 0.0
    V          (time, depth, latitude, longitude) float32 174MB 0.0 0.0 ... 0.0
    npp        (time, latitude, longitude) float32 87MB 300.0 300.0 ... 300.0
Attributes:
    comment:  Layer bounds computed from limits

## Sets primary production to constant value


In [7]:
NPP = 300
data["npp"] = data["npp"].where(data["npp"].isnull(), NPP)
data

<xarray.Dataset> Size: 610MB
Dimensions:    (latitude: 180, depth: 2, longitude: 360, time: 336)
Coordinates:
  * latitude   (latitude) float64 1kB -89.5 -88.5 -87.5 -86.5 ... 87.5 88.5 89.5
  * depth      (depth) int64 16B 0 1
  * longitude  (longitude) float64 3kB -179.5 -178.5 -177.5 ... 178.5 179.5
  * time       (time) datetime64[ns] 3kB 2000-01-16 2000-01-17 ... 2000-12-16
Data variables:
    T          (time, depth, latitude, longitude) float32 174MB 28.0 ... 28.0
    U          (time, depth, latitude, longitude) float32 174MB 0.0 0.0 ... 0.0
    V          (time, depth, latitude, longitude) float32 174MB 0.0 0.0 ... 0.0
    npp        (time, latitude, longitude) float32 87MB 300.0 300.0 ... 300.0
Attributes:
    comment:  Layer bounds computed from limits

# Dimentions order


In [8]:
data = data.transpose("time", "depth", "latitude", "longitude")

## Create mask


In [9]:
data["mask"] = xr.full_like(data["T"], 2, dtype=np.int64)
mask = data["mask"][0, 0].drop_vars(["time", "depth"])
mask.latitude.attrs = {
    "standard_name": "latitude",
    "long_name": "latitude",
    "units": "degrees_north",
    "axis": "Y",
}
mask.longitude.attrs = {
    "standard_name": "longitude",
    "long_name": "longitude",
    "units": "degrees_east",
    "axis": "X",
}
mask.attrs = {}
mask.to_netcdf(f"{export_folder}/mask.nc")
xr.load_dataset(f"{export_folder}/mask.nc")

<xarray.Dataset> Size: 523kB
Dimensions:    (latitude: 180, longitude: 360)
Coordinates:
  * latitude   (latitude) float64 1kB -89.5 -88.5 -87.5 -86.5 ... 87.5 88.5 89.5
  * longitude  (longitude) float64 3kB -179.5 -178.5 -177.5 ... 178.5 179.5
Data variables:
    mask       (latitude, longitude) int64 518kB 2 2 2 2 2 2 2 ... 2 2 2 2 2 2 2

## To float 64


In [10]:
data = data.astype(np.float64)

## Export


In [11]:
for date in data["T"].time.values:
    date_as_string = pd.to_datetime(str(date)).strftime("%Y%m%d")
    data.sel(time=[date])[["U", "V", "T", "npp"]].to_netcdf(f"{export_folder}/data_{date_as_string}.nc")

This shell commands can be run from seapodym-lmtl directory.

```bash
❯ mpirun -mca btl ^openib -np 6 ./build/bin/seapodym-lmtl -P -G D1N1 -V error /Users/ash/Documents/Workspaces/Data/phd/SEAPODYM_LMTL/2024-12-13_-_Constant_fields_no_transport/zpk.tmpl.xml
```

```bash
❯ mpirun -mca btl ^openib -np 6 ./build/bin/seapodym-lmtl -B -G D1N1 -V error /Users/ash/Documents/Workspaces/Data/phd/SEAPODYM_LMTL/2024-12-13_-_Constant_fields_no_transport/zpk.tmpl.xml
```


# TEST -------------


In [12]:
# MON FORCAGE
data

<xarray.Dataset> Size: 2GB
Dimensions:    (time: 336, depth: 2, latitude: 180, longitude: 360)
Coordinates:
  * latitude   (latitude) float64 1kB -89.5 -88.5 -87.5 -86.5 ... 87.5 88.5 89.5
  * depth      (depth) int64 16B 0 1
  * longitude  (longitude) float64 3kB -179.5 -178.5 -177.5 ... 178.5 179.5
  * time       (time) datetime64[ns] 3kB 2000-01-16 2000-01-17 ... 2000-12-16
Data variables:
    T          (time, depth, latitude, longitude) float64 348MB 28.0 ... 28.0
    U          (time, depth, latitude, longitude) float64 348MB 0.0 0.0 ... 0.0
    V          (time, depth, latitude, longitude) float64 348MB 0.0 0.0 ... 0.0
    npp        (time, latitude, longitude) float64 174MB 300.0 300.0 ... 300.0
    mask       (time, depth, latitude, longitude) float64 348MB 2.0 2.0 ... 2.0
Attributes:
    comment:  Layer bounds computed from limits

In [13]:
# LE FORCAGE ANCIEN
xr.load_dataset(
    "/Users/ash/Documents/Workspaces/Data/configurations_seapodym/lmtl/4_data_integrated/4_2_epi_upmeso_daily/integrated_forcing_20000101.nc"
)

<xarray.Dataset> Size: 3MB
Dimensions:      (latitude: 180, depth: 2, longitude: 360, time: 1,
                  limits_name: 3)
Coordinates:
  * latitude     (latitude) float64 1kB -89.5 -88.5 -87.5 ... 87.5 88.5 89.5
  * depth        (depth) int64 16B 0 1
  * longitude    (longitude) float64 3kB -179.5 -178.5 -177.5 ... 178.5 179.5
  * time         (time) datetime64[ns] 8B 2000-01-16T12:00:00
  * limits_name  (limits_name) <U23 276B 'limit_surface' ... 'limit_upper_mes...
Data variables:
    temperature  (time, depth, latitude, longitude) float32 518kB nan ... -0....
    u            (time, depth, latitude, longitude) float32 518kB nan ... -0....
    v            (time, depth, latitude, longitude) float32 518kB nan ... 0.0...
    limits       (time, limits_name, latitude, longitude) float32 778kB nan ....
    npp          (time, latitude, longitude) float32 259kB nan nan ... 0.0 0.0
Attributes:
    comment:  Layer bounds computed from limits

In [14]:
# MASK ANCIEN
xr.load_dataset(
    "/Users/ash/Documents/Workspaces/Data/configurations_seapodym/lmtl/mask/from_julien/layers_mask_corrected.nc"
)

<xarray.Dataset> Size: 132kB
Dimensions:    (latitude: 180, longitude: 360)
Coordinates:
  * latitude   (latitude) float32 720B -89.5 -88.5 -87.5 ... 87.5 88.5 89.5
  * longitude  (longitude) float32 1kB -179.5 -178.5 -177.5 ... 178.5 179.5
Data variables:
    mask       (latitude, longitude) int16 130kB 0 0 0 0 0 0 0 ... 3 3 3 3 3 3 3
Attributes:
    Title:        Pelagic layers mask for SEAPODYM model
    Description:  integer value L indicates that L layers are available at th...
    Institution:  CLS
    References:   http://www.seapodym.eu; http://www.cls.fr
    History:      Created on 2022-10-27
    Convention:   CF 1.7

In [15]:
# LE RECRUTEMENT ANCIEN
prod = xr.open_mfdataset(
    # "/Users/ash/Documents/Workspaces/Data/phd/SEAPODYM_LMTL/2024-12-13_-_Constant_fields_no_transport/output/*.nc"
    "/Users/ash/Documents/Workspaces/Data/configurations_seapodym/lmtl/output/opti_lambda/*.nc"
)
prod.load()

<xarray.Dataset> Size: 146kB
Dimensions:    (time: 1, latitude: 100, longitude: 360)
Coordinates:
  * time       (time) datetime64[ns] 8B 2000-12-16T12:00:00
  * latitude   (latitude) float32 400B -49.5 -48.5 -47.5 ... 47.5 48.5 49.5
  * longitude  (longitude) float32 1kB -179.5 -178.5 -177.5 ... 178.5 179.5
Data variables:
    zpk_epi    (time, latitude, longitude) float32 144kB 0.1413 ... 0.1413
Attributes: (12/19)
    Conventions:                   CF-1.7
    comment:                       sample
    diffusion_coefficient:         0
    domain:                        global
    energy_transfer:               0.1668
    institution:                   Mercator
    ...                            ...
    source_physical_variables:     Mercator GLORYS12V1
    title:                         Gobal ocean mid trophic levels biomass (zo...
    tr_max:                        10
    tr_rate:                       -0.11
    Created:                       2024-12-17
    date_field:                    20001216

In [16]:
# MON RECRUTEMENT
prod = xr.open_mfdataset(
    "/Users/ash/Documents/Workspaces/Data/phd/SEAPODYM_LMTL/2024-12-13_-_Constant_fields_no_transport/output/*.nc"
)
prod.load()

OSError: no files to open